In [4]:
import os
from sqlalchemy import create_engine, text
import pandas as pd

# Crear conexión con la base de datos usando variables de entorno
engine = create_engine(
    f"mysql+mysqlconnector://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)


In [10]:
# Contrataciones por seniority
q_contrataciones_por_seniority = text("""
SELECT s.level AS seniority, COUNT(*) AS contrataciones
FROM fact_application f
JOIN dim_seniority s ON s.seniority_sk = f.seniority_sk
WHERE f.hired = 1 AND s.level IS NOT NULL
GROUP BY s.level
ORDER BY contrataciones DESC;
""")

# País con más y menos personas contratadas
q_pais_mas_menos = text("""
SELECT 'Contratados'    AS tipo, d.name AS pais, t1.total AS total
FROM (
  SELECT country_sk, COUNT(*) AS total
  FROM fact_application
  WHERE hired = 1
  GROUP BY country_sk
  ORDER BY total DESC
  LIMIT 1
) t1
JOIN dim_country d ON t1.country_sk = d.country_sk

UNION ALL

SELECT 'No Contratados' AS tipo, d.name AS pais, t2.total AS total
FROM (
  SELECT country_sk, COUNT(*) AS total
  FROM fact_application
  WHERE hired = 0
  GROUP BY country_sk
  ORDER BY total DESC
  LIMIT 1
) t2
JOIN dim_country d ON t2.country_sk = d.country_sk;
""")

# Contrataciones por rango de experiencia
q_contrataciones_por_rango = text("""
SELECT
  CASE
    WHEN fa.yoe BETWEEN 0 AND 2 THEN 'Junior (0–2)'
    WHEN fa.yoe BETWEEN 3 AND 5 THEN 'SemiSenior (3–5)'
    WHEN fa.yoe >= 6              THEN 'Senior (6+)'
    ELSE 'No definido'
  END AS rango_experiencia,
  COUNT(*) AS contrataciones
FROM fact_application fa
WHERE fa.hired = 1
GROUP BY rango_experiencia
ORDER BY contrataciones DESC;
""")

# (N1) Contrataciones por año (usa dim_date.year)
q_contrataciones_por_anio = text("""
SELECT
  d.year AS anio,
  COUNT(*) AS contrataciones
FROM fact_application fa
JOIN dim_date d ON d.date_sk = fa.date_sk
WHERE fa.hired = 1
GROUP BY d.year
ORDER BY anio;
""")

# (N2) Cantidad de contratados en los últimos 12 meses (total)
q_total_ultimo_anio = text("""
SELECT
  COUNT(*) AS contrataciones_2022
FROM fact_application fa
JOIN dim_date d ON d.date_sk = fa.date_sk
WHERE fa.hired = 1
  AND d.year = 2022;
""")



# (N3) Contrataciones por tecnología (total)
q_contrataciones_por_tecnologia = text("""
SELECT
  t.name AS tecnologia,
  COUNT(*) AS contrataciones
FROM fact_application fa
JOIN dim_technology t ON t.tech_sk = fa.tech_sk
WHERE fa.hired = 1
GROUP BY t.name
ORDER BY contrataciones DESC;
""")




In [11]:

with engine.begin() as conn:
    # Originales
    df_seniority = pd.read_sql(q_contrataciones_por_seniority, conn)
    df_paises    = pd.read_sql(q_pais_mas_menos, conn)
    df_rangos    = pd.read_sql(q_contrataciones_por_rango, conn)

    # Agregadas
    df_anio      = pd.read_sql(q_contrataciones_por_anio, conn)
    df_total12m  = pd.read_sql(q_total_ultimo_anio, conn)
 
    df_tech      = pd.read_sql(q_contrataciones_por_tecnologia, conn)
   

# Vista rápida
df_seniority.head(), df_paises, df_rangos.head(), df_anio, df_total12m,  df_tech.head()


(   seniority  contrataciones
 0     Intern             989
 1     Junior             982
 2    Trainee             977
 3  Architect             976
 4     Senior             947,
              tipo                      pais  total
 0     Contratados  Northern Mariana Islands     44
 1  No Contratados                    Malawi    222,
   rango_experiencia  contrataciones
 0       Senior (6+)            5505
 1  SemiSenior (3–5)             654
 2      Junior (0–2)             576,
    anio  contrataciones
 0  2018            1414
 1  2019            1531
 2  2020            1497
 3  2021            1494
 4  2022             799,
    contrataciones_2022
 0                  799,
                   tecnologia  contrataciones
 0           Game Development             524
 1                     DevOps             498
 2      System Administration             297
 3   Adobe Experience Manager             286
 4  Development - CMS Backend             285)